TODO für Benchmark
* Multibertmodelle für mnli/imdb/sst2/ag_news finetunen
    * erst freezen der Bert layer, später unfreezen?
    * gleiche Setup wie im orig Bert  repo?
    * https://huggingface.co/Jeevesh8 hat mega viele finetunes der multiberts
* Welcher Token wird für bert classification verwendet? (Wo ist CLS?)
    * A: https://github.com/google-research/bert/blob/master/modeling.py#L234 nimmt Token 0, HF auch so. Aber erst wird die normale rep noch durch linear layer mit tanh geschickt, bevor der classifier drauf acted.
* Inputs von mnli
* Compare effect of single token vs all tokens




In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import datasets
from typing import Optional

def get_dataset(
    dataset_name: str, config: Optional[str] = None
) -> datasets.dataset_dict.DatasetDict:
    ds = datasets.load_dataset(dataset_name, config)
    assert isinstance(ds, datasets.dataset_dict.DatasetDict)
    return ds

dataset = get_dataset("glue", "mnli")

In [4]:
dataset = get_dataset("sst2")
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [7]:
dataset["validation"]

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 872
})

In [2]:
dataset["train"][0]


{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

In [6]:
from transformers import AutoTokenizer

modelname = "google/multiberts-seed_0"
tokenizer =  AutoTokenizer.from_pretrained(modelname)
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [7]:
# https://github.com/google-research/bert/blob/eedf5716ce1268e56f0a50264a88cafad334ac61/run_classifier.py#L408C3-L425C36
# The convention in BERT is:
# (a) For sequence pairs:
#  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
#  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
# (b) For single sequences:
#  tokens:   [CLS] the dog is hairy . [SEP]
#  type_ids: 0     0   0   0  0     0 0
#
# Where "type_ids" are used to indicate whether this is the first
# sequence or the second sequence. The embedding vectors for `type=0` and
# `type=1` were learned during pre-training and are added to the wordpiece
# embedding vector (and position vector). This is not *strictly* necessary
# since the [SEP] token unambiguously separates the sequences, but it makes
# it easier for the model to learn the concept of sequences.
#
# For classification tasks, the first vector (corresponding to [CLS]) is
# used as the "sentence vector". Note that this only makes sense because
# the entire model is fine-tuned.
# TODO: hier weiter machen mit example creation und finetuning tutorial folgen


def tokenize_function(examples):
    # Padding with max length 128 and always padding to that length is identical to the
    # original BERT repo. Truncation also removes token from the longest sequence one by one
    return tokenizer(
        text=examples["premise"],
        text_pair=examples["hypothesis"],
        padding="max_length",
        max_length=128,
        truncation=True,
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

In [11]:
tokenized_dataset["train"]

Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 392702
})

In [20]:
from transformers import AutoModelForSequenceClassification
import torch

torch.manual_seed(123456789)

model = AutoModelForSequenceClassification.from_pretrained("google/multiberts-seed_0", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/multiberts-seed_0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
len(dataset["train"]) / 128 * 3

9203.953125

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="../outputs/hf_ft_tutorial",
    warmup_ratio=0.1,
    evaluation_strategy="steps",
    eval_steps=1000,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128
)

In [23]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metric(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [24]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],  # type: ignore
    eval_dataset={
        "val_matched": tokenized_dataset["validation_matched"], # type: ignore
        "val_mismatched": tokenized_dataset["validation_mismatched"],
    },
)

In [25]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/root/LLM-comparison/outputs/hf_ft_tutorial/checkpoint-9000")
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
import transformers

transformers.AutoTokenizer.from_pretrained("/root/LLM-comparison/outputs/hf_ft_tutorial/checkpoint-9000")

OSError: Can't load tokenizer for '/root/LLM-comparison/outputs/hf_ft_tutorial/checkpoint-9000'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/root/LLM-comparison/outputs/hf_ft_tutorial/checkpoint-9000' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.